In [1]:
# importing necessary libraries
import scipy.io
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from scipy.signal import hilbert
from pyhht.visualization import plot_imfs
from numpy import asarray
from tqdm import tqdm
import re
from PIL import Image
import time
import pywt
import pyhht
from pyhht import EMD

In [2]:
data_names = []
for i in range(1, 20):
    my_string = 's'+str(i)+'D.mat'
    data_names.append(my_string)

In [3]:
# specified data and label directory
data_dir = '/home/kiara/Desktop/EEG_Project1/dis_data/'
label_dir = '/home/kiara/Desktop/EEG_Project1/dis_label/'
# specifiying file names for label
label_names = data_names # data and label has the same name

In [4]:
# list of training data
print("Training Data is:")
print(data_names)
print("\nTraining Label is:")
print(label_names)

Training Data is:
['s1D.mat', 's2D.mat', 's3D.mat', 's4D.mat', 's5D.mat', 's6D.mat', 's7D.mat', 's8D.mat', 's9D.mat', 's10D.mat', 's11D.mat', 's12D.mat', 's13D.mat', 's14D.mat', 's15D.mat', 's16D.mat', 's17D.mat', 's18D.mat', 's19D.mat']

Training Label is:
['s1D.mat', 's2D.mat', 's3D.mat', 's4D.mat', 's5D.mat', 's6D.mat', 's7D.mat', 's8D.mat', 's9D.mat', 's10D.mat', 's11D.mat', 's12D.mat', 's13D.mat', 's14D.mat', 's15D.mat', 's16D.mat', 's17D.mat', 's18D.mat', 's19D.mat']


In [5]:
# defining variables
# choosing a file for data extraction
file_name1 = data_names[18]
print(file_name1)

s19D.mat


In [6]:
# extracting the data
load_file1 = os.path.join(data_dir, file_name1)
data1 = scipy.io.loadmat(load_file1)
data = data1['DATA']
data = data.T

In [7]:
# extracting the label
load_file2 = os.path.join(label_dir, file_name1)
data2 = scipy.io.loadmat(load_file2)
label = data2['dist_label']

In [8]:
# generating a random number
m = re.search('s(.+?)D', file_name1)
random_state_value = int(m.group(1))

In [9]:
def wavelet_transform(input_data):
    coeff, freq = pywt.cwt(input_data, data.shape[0], "morl")
    return coeff

my_X = wavelet_transform(data)
x_train = my_X[0].reshape(my_X.shape[1], my_X.shape[2], my_X.shape[3])
x_train.shape

(48, 2828, 64)

In [10]:
'''# Wavelet Transform
def wavelet_transform(input_data):
    coeffs2 = pywt.dwt2(input_data, 'bior1.3')
    #coeffs2 = pywt.cwt(input_data)
    LL, (LH, HL, HH) = coeffs2
    return LL, LH, HL, HH
LL, LH, HL, HH = wavelet_transform(data)

z = np.concatenate((LL, LH), axis = 1)
z_dash = np.concatenate((HL, HH), axis = 1)
my_X = (np.concatenate((z.T, z_dash.T), axis=0)).T
my_X.shape'''

"# Wavelet Transform\ndef wavelet_transform(input_data):\n    coeffs2 = pywt.dwt2(input_data, 'bior1.3')\n    #coeffs2 = pywt.cwt(input_data)\n    LL, (LH, HL, HH) = coeffs2\n    return LL, LH, HL, HH\nLL, LH, HL, HH = wavelet_transform(data)\n\nz = np.concatenate((LL, LH), axis = 1)\nz_dash = np.concatenate((HL, HH), axis = 1)\nmy_X = (np.concatenate((z.T, z_dash.T), axis=0)).T\nmy_X.shape"

In [11]:
# performing train test split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(x_train, label, test_size=0.2, random_state = random_state_value)

# checking training and testing data shape
print(X_Train.shape)
print(Y_Train.shape)
print(X_Test.shape)
print(Y_Test.shape)

(38, 2828, 64)
(38, 1)
(10, 2828, 64)
(10, 1)


In [12]:
# Defining the model
modelone = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(X_Train.shape[1], X_Train.shape[2])),
    tf.keras.layers.Dense(64, activation=tf.nn.tanh),
    tf.keras.layers.Dense(64,activation=tf.nn.tanh),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

In [13]:
#opt = tf.keras.optimizers.Adam(learning_rate=0.01)
opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)
modelone.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics='acc')
modelone.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 180992)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                11583552  
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 11,587,777
Trainable params: 11,587,777
Non-trainable params: 0
_________________________________________________________________


In [14]:
# initializing log directory and model name
log_directory = 'Log_w_p'+str(m.group(1))
model_name = 'model_w_p'+str(m.group(1))+'.h5'

In [15]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 500, monitor = 'val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir = log_directory)
]

In [16]:
# model training
for i in range(0, 4):
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(x_train, label, test_size=0.2, random_state = random_state_value+i)
    results = modelone.fit(X_Train, Y_Train, validation_split = 0.2, batch_size = 4, epochs = 500, verbose=1, callbacks=callbacks)
modelone.save(model_name)

Epoch 1/500
8/8 [==============================] - 1s 56ms/step - loss: 1.2658 - acc: 0.5000 - val_loss: 0.7299 - val_acc: 0.6250
Epoch 2/500
8/8 [==============================] - 0s 8ms/step - loss: 0.6311 - acc: 0.6667 - val_loss: 0.6882 - val_acc: 0.7500
Epoch 3/500
8/8 [==============================] - 0s 8ms/step - loss: 0.5118 - acc: 0.6667 - val_loss: 0.7096 - val_acc: 0.6250
Epoch 4/500
8/8 [==============================] - 0s 8ms/step - loss: 0.4988 - acc: 0.7333 - val_loss: 0.7495 - val_acc: 0.8750
Epoch 5/500
8/8 [==============================] - 0s 8ms/step - loss: 0.4968 - acc: 0.7333 - val_loss: 0.7443 - val_acc: 0.8750
Epoch 6/500
8/8 [==============================] - 0s 8ms/step - loss: 0.4316 - acc: 0.8333 - val_loss: 0.7150 - val_acc: 0.6250
Epoch 7/500
8/8 [==============================] - 0s 8ms/step - loss: 0.4017 - acc: 0.8000 - val_loss: 0.7684 - val_acc: 0.7500
Epoch 8/500
8/8 [==============================] - 0s 8ms/step - loss: 0.2617 - acc: 0.9333 - va

8/8 [==============================] - 0s 7ms/step - loss: 0.0706 - acc: 0.9667 - val_loss: 0.9315 - val_acc: 0.6250
Epoch 65/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0679 - acc: 0.9667 - val_loss: 1.0956 - val_acc: 0.7500
Epoch 66/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0475 - acc: 0.9667 - val_loss: 1.0793 - val_acc: 0.6250
Epoch 67/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0768 - acc: 0.9333 - val_loss: 1.0158 - val_acc: 0.7500
Epoch 68/500
8/8 [==============================] - 0s 8ms/step - loss: 0.0656 - acc: 0.9333 - val_loss: 1.3770 - val_acc: 0.7500
Epoch 69/500
8/8 [==============================] - 0s 8ms/step - loss: 0.0668 - acc: 0.9667 - val_loss: 1.4166 - val_acc: 0.7500
Epoch 70/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0594 - acc: 0.9667 - val_loss: 1.3003 - val_acc: 0.7500
Epoch 71/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0623 - acc: 0.9667 - val_loss

Epoch 127/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0695 - acc: 0.9333 - val_loss: 1.4657 - val_acc: 0.7500
Epoch 128/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0652 - acc: 0.9667 - val_loss: 1.3880 - val_acc: 0.6250
Epoch 129/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0730 - acc: 0.9667 - val_loss: 1.2430 - val_acc: 0.7500
Epoch 130/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0609 - acc: 0.9667 - val_loss: 1.3606 - val_acc: 0.7500
Epoch 131/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0627 - acc: 0.9333 - val_loss: 1.4814 - val_acc: 0.7500
Epoch 132/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0671 - acc: 0.9667 - val_loss: 1.3930 - val_acc: 0.7500
Epoch 133/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0683 - acc: 0.9333 - val_loss: 1.3811 - val_acc: 0.7500
Epoch 134/500
8/8 [==============================] - 0s 8ms/step - loss: 0.0690 - a

8/8 [==============================] - 0s 7ms/step - loss: 0.0740 - acc: 0.9667 - val_loss: 1.4383 - val_acc: 0.7500
Epoch 190/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0497 - acc: 0.9667 - val_loss: 1.3706 - val_acc: 0.7500
Epoch 191/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0647 - acc: 0.9667 - val_loss: 1.2843 - val_acc: 0.7500
Epoch 192/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0650 - acc: 0.9667 - val_loss: 1.3448 - val_acc: 0.7500
Epoch 193/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0470 - acc: 0.9667 - val_loss: 1.3130 - val_acc: 0.7500
Epoch 194/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0737 - acc: 0.9333 - val_loss: 1.3045 - val_acc: 0.7500
Epoch 195/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0694 - acc: 0.9667 - val_loss: 1.3566 - val_acc: 0.7500
Epoch 196/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0691 - acc: 0.9667 - v

8/8 [==============================] - 0s 7ms/step - loss: 0.0723 - acc: 0.9333 - val_loss: 1.2505 - val_acc: 0.7500
Epoch 252/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0655 - acc: 0.9667 - val_loss: 1.2760 - val_acc: 0.7500
Epoch 253/500
8/8 [==============================] - 0s 8ms/step - loss: 0.0662 - acc: 0.9667 - val_loss: 1.2772 - val_acc: 0.7500
Epoch 254/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0693 - acc: 0.9333 - val_loss: 1.2901 - val_acc: 0.7500
Epoch 255/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0708 - acc: 0.9333 - val_loss: 1.3100 - val_acc: 0.7500
Epoch 256/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0672 - acc: 0.9333 - val_loss: 1.3609 - val_acc: 0.7500
Epoch 257/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0651 - acc: 0.9667 - val_loss: 1.2794 - val_acc: 0.7500
Epoch 258/500
8/8 [==============================] - 0s 8ms/step - loss: 0.0658 - acc: 0.9333 - v

...

8/8 [==============================] - 0s 7ms/step - loss: 0.0632 - acc: 0.9667 - val_loss: 2.4816e-05 - val_acc: 1.0000
Epoch 215/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0652 - acc: 0.9667 - val_loss: 2.2583e-05 - val_acc: 1.0000
Epoch 216/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0663 - acc: 0.9667 - val_loss: 2.5458e-05 - val_acc: 1.0000
Epoch 217/500
8/8 [==============================] - 0s 8ms/step - loss: 0.0664 - acc: 0.9667 - val_loss: 2.2145e-05 - val_acc: 1.0000
Epoch 218/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0651 - acc: 0.9667 - val_loss: 2.5254e-05 - val_acc: 1.0000
Epoch 219/500
8/8 [==============================] - 0s 8ms/step - loss: 0.0678 - acc: 0.9667 - val_loss: 2.2559e-05 - val_acc: 1.0000
Epoch 220/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0711 - acc: 0.9667 - val_loss: 2.5201e-05 - val_acc: 1.0000
Epoch 221/500
8/8 [==============================] - 0s 8ms/step - lo

Epoch 275/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0683 - acc: 0.9333 - val_loss: 3.0595e-05 - val_acc: 1.0000
Epoch 276/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0712 - acc: 0.9667 - val_loss: 2.3075e-05 - val_acc: 1.0000
Epoch 277/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0600 - acc: 0.9333 - val_loss: 2.3123e-05 - val_acc: 1.0000
Epoch 278/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0644 - acc: 0.9667 - val_loss: 2.7150e-05 - val_acc: 1.0000
Epoch 279/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0674 - acc: 0.9667 - val_loss: 2.1516e-05 - val_acc: 1.0000
Epoch 280/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0608 - acc: 0.9667 - val_loss: 2.5159e-05 - val_acc: 1.0000
Epoch 281/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0628 - acc: 0.9667 - val_loss: 2.8946e-05 - val_acc: 1.0000
Epoch 282/500
8/8 [==============================] - 0s

8/8 [==============================] - 0s 7ms/step - loss: 0.0686 - acc: 0.9667 - val_loss: 2.6029e-05 - val_acc: 1.0000
Epoch 336/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0679 - acc: 0.9333 - val_loss: 2.5069e-05 - val_acc: 1.0000
Epoch 337/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0671 - acc: 0.9667 - val_loss: 2.9368e-05 - val_acc: 1.0000
Epoch 338/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0683 - acc: 0.9667 - val_loss: 2.6280e-05 - val_acc: 1.0000
Epoch 339/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0465 - acc: 0.9667 - val_loss: 2.8601e-05 - val_acc: 1.0000
Epoch 340/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0788 - acc: 0.9667 - val_loss: 2.6948e-05 - val_acc: 1.0000
Epoch 341/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0672 - acc: 0.9667 - val_loss: 3.0495e-05 - val_acc: 1.0000
Epoch 342/500
8/8 [==============================] - 0s 7ms/step - lo

Epoch 396/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0742 - acc: 0.9667 - val_loss: 3.1958e-05 - val_acc: 1.0000
Epoch 397/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0694 - acc: 0.9667 - val_loss: 2.8169e-05 - val_acc: 1.0000
Epoch 398/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0685 - acc: 0.9667 - val_loss: 3.2445e-05 - val_acc: 1.0000
Epoch 399/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0663 - acc: 0.9333 - val_loss: 3.3545e-05 - val_acc: 1.0000
Epoch 400/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0680 - acc: 0.9667 - val_loss: 2.8556e-05 - val_acc: 1.0000
Epoch 401/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0463 - acc: 0.9667 - val_loss: 2.9874e-05 - val_acc: 1.0000
Epoch 402/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0766 - acc: 0.9333 - val_loss: 3.1697e-05 - val_acc: 1.0000
Epoch 403/500
8/8 [==============================] - 0s

8/8 [==============================] - 0s 7ms/step - loss: 0.0471 - acc: 0.9667 - val_loss: 2.8810e-05 - val_acc: 1.0000
Epoch 457/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0756 - acc: 0.9667 - val_loss: 3.1767e-05 - val_acc: 1.0000
Epoch 458/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0718 - acc: 0.9333 - val_loss: 3.7850e-05 - val_acc: 1.0000
Epoch 459/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0607 - acc: 0.9667 - val_loss: 3.1808e-05 - val_acc: 1.0000
Epoch 460/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0642 - acc: 0.9333 - val_loss: 3.3708e-05 - val_acc: 1.0000
Epoch 461/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0671 - acc: 0.9333 - val_loss: 3.5076e-05 - val_acc: 1.0000
Epoch 462/500
8/8 [==============================] - 0s 7ms/step - loss: 0.0668 - acc: 0.9667 - val_loss: 2.9853e-05 - val_acc: 1.0000
Epoch 463/500
8/8 [==============================] - 0s 7ms/step - lo

In [17]:
# model evaluation on test data
w_scores = modelone.evaluate(X_Test, Y_Test, verbose=1)

1/1 [==============================] - 0s 80ms/step - loss: 3.5513e-05 - acc: 1.0000


In [18]:
# predicting test data values
Y_hat = modelone.predict(X_Test)
Y_pred = []
for item in Y_hat:
    if item>=(np.mean(Y_hat)):
        Y_pred.append(1)
    else:
        Y_pred.append(0)

In [19]:
# obtaining accuracy values
correct = 0
for i in range(len(Y_pred)):
    if Y_Test[i] == Y_pred[i]:
        correct += 1
wavelet_accuracy = correct/len(Y_pred)

In [20]:
print("Accuracy from Wavelet")
print(wavelet_accuracy)

Accuracy from Wavelet
1.0
